In [3]:
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(context='paper', font_scale=1.15)

data_dir = '../data/datathon_SC_ACN_22/'

### Data overview

In [4]:
orders_data = pd.read_csv(data_dir + 'orders.csv', delimiter=';', index_col='order_id')
orders_data.sample(n=5)

,origin_port,3pl,customs_procedures,logistic_hub,customer,product_id,units,late_order
order_id,,,,,,,,
04484f39ca53,Athens,v_004,DTP,Venlo,Athens,1654623,472,False
e2422b8c8d6a,Rotterdam,v_002,CRF,Lille,Vienna,1644308,461,False
414502b5343e,Rotterdam,v_004,DTD,Warsaw,Turin,1657063,416,False
36cd0b850947,Rotterdam,v_004,DTD,Warsaw,Paris,1683409,476,False
9f78d5cef96d,BCN,v_002,DTD,Hamburg,Rome,1668608,470,False


#### Adding shipment distance

In [15]:
cities_data = pd.read_csv(data_dir + 'cities_data.csv', delimiter=';', index_col=['city_from_name', 'city_to_name'])

In [23]:
orders_data = orders_data.join(cities_data['distance'].rename('port_to_hub'), on=['origin_port', 'logistic_hub'])
orders_data = orders_data.join(cities_data['distance'].rename('hub_to_customer'), on=['logistic_hub', 'customer'])
orders_data['total_shipping_distance'] = orders_data['port_to_hub'] + orders_data['hub_to_customer']

#### Adding material handling and weight

In [27]:
product_data = pd.read_csv(data_dir + 'product_attributes.csv', delimiter=',', index_col='product_id')
orders_data = orders_data.join(product_data, on='product_id')

### Testing for basic correlation

In [25]:
correlations = pd.get_dummies(orders_data).corr()['late_order']
correlations.sort_values(ascending=False)

late_order                 1.000000
origin_port_Athens         0.172028
units                      0.167888
total_shipping_distance    0.120177
port_to_hub                0.099854
3pl_v_001                  0.092977
logistic_hub_Lille         0.089925
hub_to_customer            0.081983
customs_procedures_DTP     0.078287
logistic_hub_Zaragoza      0.071704
logistic_hub_Bratislava    0.044310
customer_Lyon              0.043225
origin_port_ATHENAS        0.043181
customer_Paris             0.040451
customer_Marseille         0.039465
3pl_v_004                  0.036191
origin_port_Barcelona      0.033180
customer_Lisbon            0.028801
customs_procedures_DTD     0.025520
customer_Helsinki          0.023470
customer_Bordeaux          0.023087
customer_Stockholm         0.022413
customer_Porto             0.018588
logistic_hub_Rome          0.016363
customer_Copenhagen        0.014449
origin_port_BCN            0.013806
customer_Athens            0.010951
3pl_v_003                  0